In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import sys
import QuantLib as ql

sys.path.insert(0, "/home/christophe/gitlab/norm/test/scripts/python")
sys.path.insert(0, "/home/christophe/gitlab/norm/release/api/python/ct")
sys.path.insert(0, "/home/christophe/gitlab/norm/test/scripts/python/Unitaire")

import import_allocation as ia
import import_pricing as pr
import TestWREmodelingCovFiltering as cf

In [2]:
#  SW20251113_SOFR O/4.2587  matu : 2025-11-13
#19/12/2024
# floating, ANNU
# fixe ANNU : 2025
# 1 PAIEMENT
#24/1/2025  ---> 
#24/1/2026
# "startDate" : "2024-11-21",
# swap rate : 4.2587

In [3]:
#Données
quotationAMUNDI=100.016363
today = ql.Date(21, ql.December, 2024) #date de calcul
ibordates= [0,1,2,3,4,5,6,7,8,9,10,15,20,30] # 0 obligatoire pour quantlib!années par rapport à la date de calcul où on connaît les taux
iborvaleursnp= np.array([4.57,4.418842,4.588199,4.383796,4.442341,4.472094,4.487725,
                       4.603022,4.49304931,4.55046352,4.56296427,4.817077,4.969355, 4.76199173])/100 # taux
year=360
start_date = ql.Date(21, ql.November, 2024)
end_date = start_date + ql.Period(1, ql.Years) 

coupon_tenor_year=1 # year
nbcoupon = 1
fixed_rate = 4.2587 /100


In [4]:
### quantlib

In [5]:
ql.Settings.instance().evaluationDate = today

In [6]:
def nparray2listQlDates(today,dates):
    qldates = []
    for i in range(len(dates)):
        if dates[i]>=0:
            qldates.append(today+ql.Period(dates[i], ql.Years))
        else:
            print(- dates[i])
            qldates.append(today-ql.Period(-dates[i], ql.Years))
    return qldates

In [7]:
ibordatesql2 = nparray2listQlDates(today,ibordates)
iborvaleurs=iborvaleursnp.tolist()
sofr_curve = ql.ZeroCurve(
    ibordatesql2,
    iborvaleurs,
    ql.Actual365Fixed()
)
sofr_curve
ncurve = len(ibordates)

In [8]:
ibordatesql2 

[Date(21,12,2024),
 Date(21,12,2025),
 Date(21,12,2026),
 Date(21,12,2027),
 Date(21,12,2028),
 Date(21,12,2029),
 Date(21,12,2030),
 Date(21,12,2031),
 Date(21,12,2032),
 Date(21,12,2033),
 Date(21,12,2034),
 Date(21,12,2039),
 Date(21,12,2044),
 Date(21,12,2054)]

In [9]:
iborvaleurs

[0.045700000000000005,
 0.04418842,
 0.045881990000000004,
 0.04383796,
 0.044423409999999997,
 0.04472094,
 0.04487725,
 0.046030220000000004,
 0.044930493099999996,
 0.045504635200000004,
 0.0456296427,
 0.04817077,
 0.04969355,
 0.0476199173]

In [10]:
[today, today + ql.Period(50, ql.Years)]

[Date(21,12,2024), Date(21,12,2074)]

In [11]:
sofr_handle = ql.YieldTermStructureHandle(sofr_curve)
sofr = ql.Sofr(sofr_handle)

In [12]:
d = start_date
while d <= today:
    if sofr.isValidFixingDate(d):
        sofr.addFixing(d, fixed_rate)
    d += 1

In [13]:

calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
convention = ql.Following
rule = ql.DateGeneration.Forward
end_of_month = False
coupon_tenor = ql.Period(coupon_tenor_year, ql.Years)
fixed_day_counter = sofr.dayCounter()

schedule = ql.Schedule(
    start_date,
    end_date,
    coupon_tenor,
    calendar,
    convention,
    convention,
    rule,
    end_of_month,
)
swap = ql.OvernightIndexedSwap(
    ql.Swap.Payer,
    100,
    schedule,
    fixed_rate,
    fixed_day_counter,
    sofr,
)

In [14]:
swap.setPricingEngine(
    ql.DiscountingSwapEngine(sofr_handle)
)

In [15]:
fixed_rate

0.042587

In [16]:
swap.NPV()

0.19379027646594604

In [17]:
swap.fixedLegNPV()

-4.145762583827385

In [18]:
swap.overnightLegNPV()

4.339552860293331

In [19]:
ratio=100*abs(swap.overnightLegNPV())/abs(swap.fixedLegNPV())
ratio

104.6744180967314

In [20]:
data = []
for cf in swap.fixedLeg():
    coupon = ql.as_fixed_rate_coupon(cf)
    data.append(
        (
            coupon.date(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
        )
    )

In [21]:
pd.DataFrame(
    data, columns=["date", "rate", "tenor", "amount"]
).style.format(
    {"amount": "{:.2f}", "rate": "{:.2%}"}
)

,date,rate,tenor,amount
0,"November 21st, 2025",4.26%,1.013889,4.32


In [22]:
data = []
for cf in swap.overnightLeg():
    coupon = ql.as_floating_rate_coupon(cf)
    print(coupon.date())
    print(coupon.accrualPeriod())
    print(coupon.amount())    
    data.append(
        (
            coupon.date(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
        )
    )
data

November 21st, 2025
1.0138888888888888
4.519682907978351


[(Date(21,11,2025),
  0.044577694434854966,
  1.0138888888888888,
  4.519682907978351)]

In [23]:
pd.DataFrame(
    data, columns=["date", "rate", "tenor", "amount"]
).style.format(
    {"amount": "{:.2f}", "rate": "{:.2%}"}
)

,date,rate,tenor,amount
0,"November 21st, 2025",4.46%,1.013889,4.52


In [24]:
swap.fairRate()

0.04457769443485499

In [25]:
###  wre
#  SW20251113_SOFR O/4.2587  matu : 2025-11-13
#19/12/2024
# floating, ANNU
# fixe ANNU : 2025
# 1 PAIEMENT
#24/1/2025  ---> 
#24/1/2026

pricing = pr.Pricing(pathlib='/home/christophe/gitlab/norm/release/lib/linux64/')

ibordatesnp= np.array(ibordates).astype(float)


#  actu :
m=nbcoupon
decal = today-start_date
# t1=np.array([0,year,2*year, 3*year, 4*year])/252.0
t1=np.zeros(m)
t2=np.zeros(m)
t1[0]=0.0/year # today
t2[0]=(year-decal)/year #dates of coupons  ## a generaliser !!
f12=np.zeros(m)
info = np.zeros(1,dtype='int64')
actrat=np.zeros(m)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, ibordatesnp, iborvaleursnp, m, t1, t2, interpmethod, actrat, info )

Loading Pricing module:
Pricing module loaded.
 r1=   4.5700000000000005E-002
 r2=   4.4314385000000005E-002
 f12=   4.4314385000000005E-002


In [26]:
t1

array([0.])

In [27]:
# pate flottante floatrate :
mfl=1
t1fl=np.zeros(mfl)
t2fl=np.zeros(mfl)
t1fl[0]=np.array([0.])/year
t2fl[0]=(year-decal)/year #dates of coupons  ## a generaliser !!
info = np.zeros(1,dtype='int64')
floatrate=np.zeros(mfl)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, ibordatesnp, iborvaleursnp, mfl, t1fl, t2fl, interpmethod, floatrate, info )

 r1=   4.5700000000000005E-002
 r2=   4.4314385000000005E-002
 f12=   4.4314385000000005E-002


/tmp/ipykernel_530212/2549259211.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  t1fl[0]=np.array([0.])/year


In [28]:
# jambe flottante
ndatesFL = m
datesFL = t2
actualizationratesFL = actrat/100.0
principalFL = 100.0
yearfractionFL = 1.0
floatingrate = floatrate

# jambe fixe
ndatesFIX = m
datesFIX = t2
actualizationratesFIX = actrat/100.0
principalFIX = 100.0
yearfractionFIX = 1.0
fixedrate = 4.2587/100.0
buyer=0

#valo
fixedleg = np.zeros(1)
floatingleg = np.zeros(1)
price = np.zeros(1)
pricing.WREpricingIRS2( ndatesFL,  datesFL,  actualizationratesFL,   principalFL,  yearfractionFL, floatingrate,
                      ndatesFIX, datesFIX, actualizationratesFIX, principalFIX, yearfractionFIX, 
                      fixedrate, buyer, fixedleg, floatingleg, price, info)
ratioWRE = 100*floatingleg[0]/fixedleg[0] 
ratioWRE

 principal actualized from fixed leg=   99.959386729838883     
   100.00000000000000        4.4314385000000003E-004  0.91666666666666663     
 fixedleg=   4.2569704026636490     
 principal actualized from floating leg=   99.959386729838883     
 test   100.00000000000000        4.4314385000000003E-004  0.91666666666666663                1
 floatingleg=   4.4296387479099719     
 fixedleg=   4.2569704026636490     
 floatingleg=   4.4296387479099719     
 P=  0.17266834524632291     


np.float64(104.05613215300444)

In [29]:
###### Comparaison

In [30]:
data={"Market Value": [quotationAMUNDI,"x", "x","x"],
      "WRE":[ratioWRE,price[0],fixedleg[0], floatingleg[0]],
      "Quantlib": [ratio, swap.NPV(),abs(swap.fixedLegNPV()), abs(swap.overnightLegNPV())], 
     }

pd.DataFrame(data, index=['ratio', 'value', "fixed leg", "floating leg"])


,Market Value,WRE,Quantlib
ratio,100.016363,104.056132,104.674418
value,x,0.172668,0.193790
fixed leg,x,4.256970,4.145763
floating leg,x,4.429639,4.339553
